# GitHubでmatplotlibにstarをつけている人が、他にどんなリポジトリに注目しているか調べてみる

## このページは何?

このページは**<a href="https://qiita.com/advent-calendar/2017/geekwomenjapan">「Geek Women Japan Advent Calendar 2017」</a>**の16日目の記事です。   
先月参加させて頂いた**<a href="https://pyladies-tokyo.connpass.com/event/64432/">PyLadies Tokyo 秋合宿 2017</a>**中に、   

** GitHub上でmatplotlibにstarをつけている人が、他にどんなリポジトリにstarをつけているか? **   

をPythonを使って調べてみたのですが、合宿中はデータ取得時間の関係上、matplotlibにstarをつけているユーザ約6000人に対し400人程度しか調べられませんでした。その後補習(?)として、約6000人分のデータを調べてみたので紹介します。

## 何故そんなことやったの？ 

** 理由：自分が関心のありそうなGitHubのリポジトリを見逃したくないから**
   
Pythonは便利なライブラリが沢山ありますが、**沢山ありすぎて自分が関心ありそうなものを見逃している可能性があります。**  
自分のGitHubアカウントの情報を利用することで、これまで知らなかったリポジトリを発掘できるかもしれません。   
以下のような手段がいくつか考えられますが、シンプルにできそうな方法として、今回は1をやってみることにしました。    

1. **自分がstar登録しているリポジトリに対して、他に誰がstar登録しているかリストアップし、   
   その人たちが他にどんなリポジトリにstarをつけているか調べてみる  *★今回はこれをやる* **
2. フォロー・フォロワー関係にあるGitHubユーザがstar登録しているリポジトリが何か調べてみる
3. 関心がある言語のGitHubTrendingページを監視する
4. GitHubのデータを使って推薦モデルを作る

**　※何故matplotlibなの?**   
<a href="https://github.com/komo-fr?tab=stars">私がstarをつけているリポジトリ</a>の中で、有名所かつstar数が多すぎず少なすぎずだったためです。~~ぶっちゃけ深く考えずに選んだ~~

## 対象データ

- 2017/11/25時点でmatplotlibにstarをつけていたユーザ6282人中<u>**6253人**</u>
- 上記ユーザが2017/11/26-2017/12/3 期間中にスターをつけていたリポジトリ 

大勢がざっくりわかればいいので、ゆるっと収集しています。   
全員が対象ではないのは、あまりにstar数が多くてデータ取得が終わらないユーザや、期間中にGitHubのアカウントが消えたor名前が変更されたのかNot Foundになっているユーザが存在したためです。またリポジトリデータの取得期間に幅があるのは、データ量が多くて小分けにしてちまちま集めていたことによります。   
なおデータ取得には、<a href="https://github.com/PyGithub/PyGithub">PyGithub</a>を使用しました。

## 結果
ファイル「<a href="https://github.com/komo-fr/AdventCalender/blob/master/2017/ranking_in_matplotlib_stargazers.csv">ranking_in_matplotlib_stargazers.csv</a>」に集計結果が入っています。

- ***rank:*** 「matplotlibにstarをつけていたユーザ」内でstar数が多かったリポジトリの順位。
- ***repository_name:*** リポジトリのフルネーム。   
「https://github.com/ 」の後にリポジトリのフルネームを付け加えたurlで、当該リポジトリのページに飛べます。
- ***stars_from_matplotlib_stargazers:*** 「matplotlibにstarをつけていたユーザ」からついたstar数。   

### TOP10
まずはTOP10を見てみましょう。なお1位は当然matplotlibなので、ここでは除外して表示しています。

In [2]:
import pandas as pd
# データの読み込み
ranking_df= pd.read_csv('ranking_in_matplotlib_stargazers.csv', encoding='utf-8')
# TOP10を表示する
display_count = 10
display_ranking_df = ranking_df[1:] #  1位のリポジトリ(index=0)は当然matplotlibなので、除外する
display_ranking_df['rank'] = display_ranking_df['rank'] - 1 # matplotlibが消えた分、順位をずらす
display_ranking_df = display_ranking_df.set_index('rank')
display_ranking_df.head(display_count)

,repository_name,stars_from_matplotlib_stargazers
rank,,
1,tensorflow/tensorflow,2461
2,scikit-learn/scikit-learn,2168
3,d3/d3,2023
4,pandas-dev/pandas,1863
5,twbs/bootstrap,1709
6,vinta/awesome-python,1691
7,ipython/ipython,1654
8,pallets/flask,1629
9,requests/requests,1580


tensorflow, scikit-learnと機械学習ライブラリの人気どころがきて、同じ可視化系のカテゴリのためかD3が3位に入っています。   
   
次に、「matplotlibにstarをつけていたユーザ」の内どのぐらいのユーザから支持を受けているのか、割合を計算してみます。   
ついでにpandasのstyling機能を使って、視覚的にわかりやすくしてみましょう。

**Styling:**   
https://pandas.pydata.org/pandas-docs/stable/style.html

数も増やして、TOP25まで表示します。　　　

In [3]:
display_count = 25  # TOP25まで表示する
# matplotlibにstarをつけているユーザの総数
matplotlib_stargazers_total_count = ranking_df.loc[ranking_df.repository_name=='matplotlib/matplotlib', 'stars_from_matplotlib_stargazers'].values[0]
# 割合を計算する
display_ranking_df['ratio'] = display_ranking_df.stars_from_matplotlib_stargazers / matplotlib_stargazers_total_count 
display_ranking_df.head(display_count).style.bar(subset=['ratio'], color='#FF7400')

,repository_name,stars_from_matplotlib_stargazers,ratio
rank,,,
1,tensorflow/tensorflow,2461,0.393571
2,scikit-learn/scikit-learn,2168,0.346714
3,d3/d3,2023,0.323525
4,pandas-dev/pandas,1863,0.297937
5,twbs/bootstrap,1709,0.273309
6,vinta/awesome-python,1691,0.27043
7,ipython/ipython,1654,0.264513
8,pallets/flask,1629,0.260515
9,requests/requests,1580,0.252679


** ※ 追記 **   
GitHub上だとstyle設定をかけた状態でレンダリングされない?ようなので、<a href="https://github.com/komo-fr/AdventCalender/blob/master/GeekWomenJapanAdventCalendar2017/styling_image.png">ここ</a>に画像を置いておきます。

さてここで当初の目的を思い出してみましょう。そもそもの目的は、

** 自分が関心のありそうなGitHubのリポジトリを見逃したくないから** 

です。言い換えれば、今まで見逃していたリポジトリを発掘できなければあんまり嬉しくないわけです。   
先ほどのランキングを見返してみると、上位に君臨しているのはどれも有名どころのリポジトリで大きな「びっくり」がなく、個人的には(´・ω・`)です。   
問題自体は「おむつとビール」の話っぽいので、下記ページみたいなことを勉強すればもう少し良い方法がわかるかもしれない......のですが、普段エンジニアなのでニワカ知識で変なこと書けない...... 

**商品分析の手法（ABC分析、アソシエーション分析）**   
https://www.albert2005.co.jp/knowledge/marketing/customer_product_analysis/abc_association

このままで終わるのも寂しいので、ついでに上位100位に入ったリポジトリに対して、   

** リポジトリAとmatplotlibの両方にstarをつけているユーザ数 / リポジトリAにstarをつけている全ユーザ数**

を求めた結果のランキングも見てみます。

……でも、AdventCalenderの公開締切が迫っているので、いったんここで公開します。   
※ 次はこのノートに追記するかもしれないし、別記事（というか別Jupyter Notebook)で書くかもしれない

## コメント先
コードをコピペする手間を省くためにGitHub上でJupyter Notebookを公開する形で書いたのですが、コメントとかつけにくいことに気づきました。   
なので、何かあればtwitterの<a href="https://twitter.com/komo_fr">@komo_fr</a>までお願いします。